# 四、网络爬虫模型

想要编写一个干净、可扩展的代码是很难的。为了更好的抓取和存储来自多组网站的各种各样的数据，并且这些数据是程序员无法控制的，这就带来了独特的组织挑战。

尽管网络爬虫的应用几乎是无止尽的，但大型、可扩展的爬虫往往分为几种模式。通过学习这些模式并识别它们适用的场景，你可以大幅改善你的网络爬虫的可维护性和稳健性。

## 4.1 规划和定义对象

网页抓取过程中的一个常见陷阱：完全基于眼前可见的内容定义自己希望抓取的数据。即使是同类产品的不同网站中的数据结构也会有不同之处，不同类的产品之间的结构差距就更大了。这种时候“所见即所得”的结构是一种不可持续的方法。

正确的做法是：不要查看网站中给出了什么结构，而是首先问自己，你需要什么？然后想法设法从中寻找所需信息。例如：你可能只需要比较多个商店的产品价格，并且追踪这些价格的变化。这种情况下，你需要足够的信息来唯一地识别各个产品。

除了你明确需要的信息之外，其他的信息是特定于产品的，这些信息可以单独保存。同时，这些信息可能很稀疏，因为并不是所有产品都有这个信息。

这时候，我们需要后退一步，对你考虑的每一项都做一个清单检查，然后问自己以下几个问题：

- 这个信息可以帮助项目实现目标吗？如果我不包括该信息，是否会造成阻碍？还是说该信息有了固然好，但是并不会影响任何结果？
- 如果该信息将来可能有帮助，但是我并不确定，那么晚些时候再抓取会有多大的困难？
- 这个数据对于我已经抓取的信息来说是否冗余？
- 将数据存储在这个对象中是否符合逻辑？（正如前面提到的，如果同一产品在不同网站上的描述不一致的话，那么存储该产品的描述信息就没有意义。）

如果你确定需要抓取该数据，那么就要问自己以下问题，然后确定如何在代码中存储并处理这些数据。

- 该数据是系数的还是密集的？它与每个清单都相关并且会出现在其中，还是只与部分清单相关？
- 该数据有多大？
- 在数据较大的情况下，我每次运行分析时都需要检索该数据，还是只是偶尔需要使用该数据？
- 这种类型的数据有多大的变化性？我需要经常加入新的属性、修改类型（例如面料样式可能是经常修改的属性）吗？还是说该数据一直保持不变（鞋的码数）？

加入你计划对产品属性和价格做一些元数据分析，此时，你发现数据时稀疏的，因此你可能决定经常增加或者移除部分属性。这样的话，创建一个如下所示的产品类型可能比较合理：

- 产品名称
- 制造商
- 产品ID（如果可以获得/相关）
- 属性（可选列表或字典）

其中的属性类型如下所示：

- 属性名
- 属性值

这样，你就可以灵活地添加新的产品属性，而无需重新设计数据模式或者重写代码。

决定好如何在数据库中存储这些属性后，你可以在JSON中编写attribute字段，或者将每个属性与产品ID一起存在一个单独的表格中。

当面临一个新项目时，很容易立马开始写Python代码来抓取网站。而数据模型通常是后面考虑的内容，并且通常会被你抓取的第一个网站的数据可用性和数据格式所影响。

但是数据模型是所有代码的基础。模型中糟糕的决定很容易导致代码编写和维护的问题，或者导致难以抽取和高效地使用数据。特别是当你处理很多类型的网站（包括已知的和未知的）时，认真思考并规划你究竟需要抓取什么以及如何存储变得非常关键。

## 4.2 处理不同的网站布局

机器对于识别页面的标题和主要内容的任务是非常困难的。幸运的是，大多数网页抓取任务中，你不会去抓取你从未见过的网站，而是从一些人为预选的网站中抓取。也就是，你可以手动确定网页上的各个元素。

最显而易见的方法是，为每个网站单独编写一个网络爬虫或者页面解析器。每个爬虫或解析器以一个URL、字符串或者BeautifulSoup对象作为输入，并返回一个抓取的Python对象。

In [ ]:
import requests
from bs4 import BeautifulSoup

class Content:
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body

def getPage(url):
    req = requests.get(url)
    return BeautifulSoup(req.text, 'html.parser')
    
def scrapeNYTimes(url):
    bs = getPage(url)
    title = bs.find("h1").text
    lines = bs.find_all("p", {"class": "story-content"})
    body = '\n'.join([line.text for line in lines])
    return Content(url, title, body)

def scrapeBrookings(url):
    bs = getPage(url)
    title = bs.find("h1").text
    body = bs.find("div", {"class", "post-body"}).text
    return Content(url, title, body)

url = 'https://www.brookings.edu/blog/future-development' \
    '/2018/01/26/delivering-inclusive-urban-access-3-unc' \
    'omfortable-truths/'
content = scrapeBrookings(url)
print('Title: {}'.format(content.title))
print('URL: {}'.format(content.url))
print(content.body)

url = 'https://www.nytimes.com/2018/01/25/opinion/sunday/' \
    'silicon-valley-immortality.html'
content = scrapeNYTimes(url)
print('Title: {}'.format(content.title))
print('URL: {}'.format(content.url))
print(content.body)

为了更简便，你可以不处理所有的标签参数和键/值对，而是用单个CSS选择器使用BeautifulSoup的select函数选定你希望抓取的信息，并且将这些选择器放入到一个字典对象中。

In [1]:
class Content:
    """
    所有文章/网页的共同基类
    """
    def __init__(self, url, title, body):
        self.url = url
        self.title = title
        self.body = body
    
    def print(self):
        """
        用灵活的打印函数控制结果
        """
        print("URL: {}".format(self.url))
        print("TITLE: {}".format(self.title))
        print("BODY:\n{}".format(self.body))

class Website:
    """
    描述网站结构的信息
    """

    def __init__(self, name, url, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.titleTag = titleTag
        self.bodyTag = bodyTag

注意：以上的Website类并不存储任何从页面本身抓取的信息，而是存储关于如何抓取数据的指令。他也不存储“My Page Title”这样的标题。他只会存储字符串标签h1，表明了在哪里可以找到标题。这就是这个类被命名为Website（包含适用于整个网站的信息）而不是Content（包含来自单个页面的信息）的原因

使用这些Content和Website类，你就可以编写一个Crawler去抓取任何网站的任何网页的标题和内容：

In [1]:
import requests
from bs4 import BeautifulSoup

class Crawler:
    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')

    def safeGet(self, pageObj, selector):
        """
        如果从一个BeautifulSoup对象和一个选择器获取内容的辅助函数。如果选择器没有找到对象，就返回空字符串
        """
        selectedElems = pageObj.select(selector)
        if selectedElems is not None and len(selectedElems) > 0:
            return '\n'.joint([elem.get_text() for elem in selectedElems])
        return ''

    def parse(self, site, url):
        """
        从指定URL提取内容
        """
        bs = self.getPage(url)
        if bs is not None:
            title = self.safeGet(bs, site.titleTag)
            body = self.safeGet(bs, site.bodyTag)
        if title != '' and body != '':
            content = Content(url, title, body)
            content.print()

In [ ]:
crawler = Crawler()

siteData = [
    ["O'Reilly Media", 'http://oreilly.com', 'h1', 'section#product-description'],
    ["Reuters", 'http://reuters.com', 'h1', 'div.StandardArticleBody_body_1gnLA'],
    ["Brookings", 'http://www.brookings.edu', 'h1', 'div.post-body'],
    ["New York Times", 'http://nytimes.com', 'h1', 'p.story-content']
    ]
websites = []
for row in siteData:
    websites.append(Website(row[0], row[1], row[2], row[3]))

crawler.parse(websites[0], "http://shop.oreilly.com/product/0636920028154.do")
crawler.parse(websites[1], "http://www.reuters.com/article/us-usa-epa-pruitt-idUSKBN19W2D0")
crawler.parse(websites[2], "http://www.brookings.edu/blog/techtank/2016/03/01/idea-to-retire-old-methods-of-policy-education/")
crawler.parse(websites[3], "http://www.nytimes.com/2018/01/28/business/energy-environment/oil-boom.html")

这个方法对于抓取大量的网站会更有用。

由于每个字符串列表写起来相对容易，并且也不会占用太多空间。他可以通过一个数据库或者CSV文件加载。它可以从远程源导入，或者交给一个有前端经验的非程序员来填充并加入新网站，而无须阅读代码。

不足之处再与牺牲了一定的灵活性：每个网站必须具有一定的结构，即特定的字段必须存在，从字段取出的数据必须干净，并且每个目标字段必须有唯一且可靠的CSS选择器。

但是，这种方法的强大功能和灵活性足以弥补其缺陷。

## 4.3 结构化爬虫

这里讲述如何将上面的方法应用于结构良好的、可扩展的网站爬虫，以自动搜集链接和发现数据。

以下介绍三种基本的网络爬虫结构，我认为他们可以应用于大多数情形，不过你在抓取网站时可能需要做一些改动。如果你碰到了特殊情形，遇到了抓取问题，我也希望你能借鉴这些结构来设计优雅、健壮的爬虫。

### 4.3.1 通过搜索抓取网站

抓取网站的一种最简单的方法是像人类一样使用搜索条。

尽管在网站上搜索关键词或者主题并收集搜索结果的过程，看起来是个随着网站的不同有很大可变性的任务，但有几个关键点使得这个人物出人意料地简单：

- 大多数网站通过将主题作为参数在URL中传递，来获得特定主题的搜索结果列表。例如，`http://example.com?search=myTopic`。这个URL的第一部分可以存为Website对象的一个属性，简单地在其后添加主题。
- 在搜索后，大多数网站以非常好识别的链接列表的形式呈现结果页面，通常会使用一个如`<span class="result">`的标签，而且准确的形式也可以存为Website对象的一个属性。
- 每个结果链接要么是一个相对URL（如`/articles/page.html`），要么是一个绝对URL（如`http://example.com/articles/page.html`）。不管是相对URL还是绝对URL，你都可以将其存为Website对象的一个属性。
- 当定位到并规范化搜索页面的URL后，你就成功地将问题简化为上一节示例中的问题了——抽取给定格式网站的数据。

接下来我们实现该算法：

In [2]:
class Content:
    """所有文章、网站的共同基类"""
    def __init__(self, topic, url, title, body):
        self.topic = topic
        self.url = url
        self.title = title
        self.body = body

    def print(self):
        """
        用灵活的答应函数控制结果
        """
        print(f"New article found for topic: {self.topic}")
        print(f"TITLE: {self.title}")
        print(f"BODY: {self.body}")
        print(f"URL: {self.url}")

In [3]:
class Website:
    """描述网站结构的信息"""
    def __init__(self, name, url, searchUrl, resultListing, resultUrl, absoluteUrl, titleTag, bodyTag):
        self.name = name
        self.url = url
        self.searchUrl = searchUrl
        self.resultListing = resultListing
        self.resultUrl = resultUrl
        self.absoluteUrl = absoluteUrl
        self.titleTag = titleTag
        self.bodyTag = bodyTag

In [4]:
import requests
from bs4 import BeautifulSoup

class Crawler:
    def getPage(self, url):
        try:
            req = requests.get(url)
        except requests.exceptions.RequestException:
            return None
        return BeautifulSoup(req.text, 'html.parser')

    def safeGet(self, pageObj, selector):
        childObj = pageObj.select(selector)
        if childObj is not None and len(childObj) > 0:
            return childObj[0].get_text()
        return ''

    def parse(self, topic, site):
        """
        根据主题搜索网站并记录所有找到的页面
        """
        bs = self.getPage(site.searchUrl + topic)
        searchResults = bs.select(site.resultListing)
        for result in searchResults:
            url = result.select(site.resultUrl)[0].attrs["href"]
            # 检查一下是相对URL还是绝对URL
            if(site.absoluteUrl):
                bs = self.getPage(url)
            else:
                bs = self.getPage(site.url + url)
            if bs is None:
                print("Something was wrong with that page or URL, Skipping!")
                return
            title = self.safeGet(bs, site.titleTag)
            body = self.safeGet(bs, site.bodyTag)
            if title != '' and body != '':
                content = Content(topic, title, body, url)
                content.print()

In [ ]:
crawler = Crawler()

siteData = [
    ["O'Reilly Media", "http://oreilly.com", "https://ssearch.oreilly.com/?q=", "article.product-result", "p.title a", True, "h1", "section#product-description"],
    ["Reuters", "http://reuters.com", "http://www.reuters.com/search/news?blob=", "div.search-result-content", "h3.search-result-title a", False, "h1", "div.StandardArticleBody_body_1gnLA"],
    ["Brookings", "http://www.brookings.edu", "http://www.brookings.edu/search/?s=", "div.list-content article", "h4.title a", True, "h1", "div.post-body"],
    ]
sites = []
for row in siteData:
    websites.append(Website(row[0], row[1], row[2], row[3],
                            row[4], row[5], row[6], row[7]))

topics = ['python', 'data science']
for topic in topics:
    print(f"GETTTING INFO ABOUT: {topic}")
    for targetSite in sites:
        crawler.search(topic, targetSite)